## 연습 문제

사용자 정의 도구를 직접 만들어 Llama 3.1에서 사용할 수 있도록 구성해보세요. 이러한 도구는 소프트웨어를 감싸는 래퍼 함수일 수도 있고, 섭씨를 화씨로 변환하는 등 특정 기능을 수행하는 단일 함수일 수도 있습니다.

다음의 다섯 가지 도구를 JSON 형식으로 정의하고, 시스템 프롬프트에 포함해보세요.

- Wikipedia API를 질의하는 도구
- arXiv API를 질의하는 도구
- 섭씨를 화씨로 변환하는 도구
- 입력을 텍스트 파일로 저장하는 도구
- 파일을 복사하는 도구

각 도구가 호출되도록 유도할 수 있는 질문을 입력해 테스트해보세요. 도구가 제대로 호출되지 않는다면 설명을 수정한 후 다시 시도해보세요. 이러한 설명 변경이 호출 성능 향상에 도움이 되는지 관찰해보는 것도 좋습니다


In [ ]:
# 필요한 라이브러리 설치
# !pip install langchain langchain-community langchain-openai wikipedia duckduckgo-search arxiv

In [ ]:
import os
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime

# OpenAI API 키 설정 (환경변수에서 가져오기)
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"  # 실제 키로 변경 필요

# 1. 기본 도구들 (API 키 불필요)
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
duckduckgo_tool = DuckDuckGoSearchRun()
arxiv_tool = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

# 2. 추가 커스텀 도구들 (API 키 불필요)

def calculator_func(expression: str) -> str:
    """기본 수학 계산을 수행합니다."""
    try:
        # 안전한 계산을 위해 eval 대신 제한적 연산 허용
        allowed_chars = "0123456789+-*/()."
        if all(c in allowed_chars or c.isspace() for c in expression):
            result = eval(expression)
            return f"계산 결과: {result}"
        else:
            return "잘못된 수식입니다. 숫자와 기본 연산자만 사용해주세요."
    except Exception as e:
        return f"계산 오류: {str(e)}"

def current_time_func(query: str) -> str:
    """현재 시간과 날짜를 제공합니다."""
    now = datetime.now()
    return f"현재 시간: {now.strftime('%Y년 %m월 %d일 %H시 %M분 %S초')}"

def weather_info_func(location: str) -> str:
    """간단한 날씨 정보를 제공합니다 (시뮬레이션)."""
    # 실제로는 날씨 API를 사용해야 하지만, 여기서는 시뮬레이션
    import random
    weather_conditions = ["맑음", "흐림", "비", "눈", "안개"]
    temp = random.randint(-10, 35)
    condition = random.choice(weather_conditions)
    return f"{location}의 날씨 정보 (시뮬레이션): 기온 {temp}°C, 날씨: {condition}"

# 도구들을 Tool 객체로 래핑
calculator_tool = Tool(
    name="Calculator",
    func=calculator_func,
    description="기본 수학 계산(+, -, *, /)을 수행합니다. 예: '2+3*4'"
)

time_tool = Tool(
    name="CurrentTime", 
    func=current_time_func,
    description="현재 시간과 날짜를 알려줍니다."
)

weather_tool = Tool(
    name="Weather",
    func=weather_info_func, 
    description="특정 지역의 날씨 정보를 제공합니다. 도시 이름을 입력하세요."
)

# 모든 도구들을 리스트로 정리
tools = [
    wikipedia_tool,
    duckduckgo_tool, 
    arxiv_tool,
    calculator_tool,
    time_tool,
    weather_tool
]

# 시스템 프롬프트 (도구 선택 힌트 포함)
system_prompt = """당신은 다양한 도구를 사용할 수 있는 지능형 어시스턴트입니다.

사용 가능한 도구들과 사용 시기:

1. **Wikipedia**: 일반적인 지식, 역사, 인물, 개념에 대한 정보가 필요할 때
   - 예: "아인슈타인은 누구인가?", "파이썬 프로그래밍 언어란?"

2. **DuckDuckGo**: 최신 정보, 뉴스, 실시간 검색이 필요할 때
   - 예: "2024년 AI 뉴스", "최신 기술 동향"

3. **ArXiv**: 학술 논문, 연구 자료, 특히 AI/ML 관련 논문 검색
   - 예: "transformer 모델 논문", "대화형 AI 연구"

4. **Calculator**: 수학 계산이 필요할 때
   - 예: "25 * 48은?", "복잡한 수식 계산"

5. **CurrentTime**: 현재 시간이나 날짜 정보가 필요할 때
   - 예: "지금 몇 시인가?", "오늘 날짜는?"

6. **Weather**: 날씨 정보가 필요할 때
   - 예: "서울 날씨", "부산의 기온"

**도구 선택 가이드라인:**
- 질문의 성격을 먼저 파악하세요
- 최신 정보가 필요하면 DuckDuckGo 우선 고려
- 학술적/연구 내용이면 ArXiv 검색
- 일반 지식은 Wikipedia 활용
- 수치 계산은 Calculator 사용

질문에 가장 적합한 도구를 선택하여 정확하고 유용한 답변을 제공하세요."""

# ChatPromptTemplate 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# LLM 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 에이전트 생성
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 테스트 함수
def test_agent_with_queries():
    """다양한 쿼리로 에이전트 테스트"""
    
    test_queries = [
        "아인슈타인의 상대성 이론에 대해 알려주세요",  # Wikipedia 예상
        "2024년 AI 최신 뉴스를 찾아주세요",  # DuckDuckGo 예상  
        "transformer 모델에 관한 최신 논문을 찾아주세요",  # ArXiv 예상
        "127 * 89는 얼마인가요?",  # Calculator 예상
        "지금 몇 시인가요?",  # CurrentTime 예상
        "서울의 날씨는 어떤가요?"  # Weather 예상
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*60}")
        print(f"테스트 {i}: {query}")
        print('='*60)
        
        try:
            result = agent_executor.invoke({"input": query})
            print(f"결과: {result['output']}")
        except Exception as e:
            print(f"오류 발생: {str(e)}")
        
        print("-" * 60)

# 개별 쿼리 테스트 함수
def ask_agent(question: str):
    """사용자 질문에 대해 에이전트가 답변"""
    try:
        result = agent_executor.invoke({"input": question})
        return result['output']
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

# 사용 예시
if __name__ == "__main__":
    print("LangChain 다중 도구 에이전트가 준비되었습니다!")
    print(f"사용 가능한 도구: {[tool.name for tool in tools]}")
    
    # 테스트 실행
    print("\n자동 테스트를 실행합니다...")
    test_agent_with_queries()
    
    # 대화형 모드
    print("\n\n대화형 모드를 시작합니다. 'quit'를 입력하면 종료됩니다.")
    while True:
        user_input = input("\n질문을 입력하세요: ")
        if user_input.lower() in ['quit', 'exit', '종료']:
            break
        
        print("답변을 생성중입니다...")
        response = ask_agent(user_input)
        print(f"\n답변: {response}")

# 도구별 성능 분석 함수
def analyze_tool_usage():
    """각 도구의 사용 적합성 분석"""
    print("\n=== 도구별 사용 가이드 ===")
    
    tool_guide = {
        "Wikipedia": {
            "적합한 질문": ["역사적 사실", "인물 정보", "개념 설명", "일반 지식"],
            "예시": "나폴레옹은 누구인가요?"
        },
        "DuckDuckGo": {
            "적합한 질문": ["최신 뉴스", "실시간 정보", "트렌드"],
            "예시": "오늘의 주요 뉴스는?"
        },
        "ArXiv": {
            "적합한 질문": ["학술 논문", "연구 자료", "AI/ML 논문"],
            "예시": "GPT 모델에 관한 논문을 찾아주세요"
        },
        "Calculator": {
            "적합한 질문": ["수학 계산", "수식 해결"],
            "예시": "156 * 78은 얼마인가요?"
        },
        "CurrentTime": {
            "적합한 질문": ["현재 시간", "날짜 정보"],
            "예시": "지금 몇 시인가요?"
        },
        "Weather": {
            "적합한 질문": ["날씨 정보", "기온 정보"],
            "예시": "부산 날씨가 어떤가요?"
        }
    }
    
    for tool_name, info in tool_guide.items():
        print(f"\n📱 {tool_name}")
        print(f"   적합한 질문 유형: {', '.join(info['적합한 질문'])}")
        print(f"   예시: \"{info['예시']}\"")

# 분석 실행
analyze_tool_usage()

print("\n=== 개선 사항 제안 ===")
improvements = [
    "1. 더 구체적인 시스템 프롬프트로 도구 선택 정확도 향상",
    "2. 도구 실행 결과에 대한 후처리 및 검증 로직 추가", 
    "3. 사용자 피드백을 통한 도구 선택 학습 메커니즘",
    "4. 복합 질문에 대한 다중 도구 연계 사용 최적화",
    "5. 도구별 응답 시간 모니터링 및 성능 개선"
]

for improvement in improvements:
    print(improvement)